# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом разрабатывает приложение для определения рыночной стоимости своего автомобиля. Нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# Описание данных
# Признаки
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- NotRepaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

# Целевой признак
- Price — цена (евро)

# Инструкция по выполнению проекта
Чтобы усилить исследование, не ограничивайтесь градиентным бустингом. Попробуйте более простые модели — иногда они работают лучше. Это редкие случаи, которые легко пропустить, если всегда применять только бустинг. Поэкспериментируйте и сравните характеристики моделей: скорость работы, точность результата.

## Подготовка данных

In [ ]:
!pip install -U scikit-learn

     |████████████████████████████████| 22.3MB 181kB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
!pip install catboost

     |████████████████████████████████| 69.2MB 65kB/s 


In [ ]:
import numpy as np
import pandas as pd
import time

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:

df.head(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [ ]:
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [ ]:
df.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Имеем массив данных размером 354тыс строк на 16 столбцов:
- в столбце VehicleType имеется 37,5тыс пропусков;
- в столбцах Gearbox и Model почти по 20тыс пропусков;
- в столбце FuelType имеется почти 33тыс пропусков;
- в столобце NotRepaired - 71тыс пропусков;
- в столбце Price есть строки с 0 стоимостью машины - это наш целевой признак, пропусков нет;
- в столбце RegistrationYear имеются строки с годом 9999;
- в столбце RegistrationMonth можно удалить, тк для возраста важен год регистрации;
- в столбце Power не везде указана мощность моторов + есть записи в 20тыс л.с., что явно либо опечатка, либо аномалия;
- в столбце NumberOfPictures нет ни одной фотографии, можно сразу удалить.


## Предобработка данных

Удалим столбцы с фотографиями, потому что их нет; 
а также столбецы с почтовым индексом и с месяцем регистарции,
тк он не несет полезной информации: важен год регистрации для определея возраста машины

In [ ]:
df = df.drop(['NumberOfPictures', 'RegistrationMonth', 'PostalCode'], axis = 1)

In [ ]:
# Так как время и день внесения в систему новой позиции нам не важны, то столбцы можно удалить.
df = df.drop(['DateCrawled','DateCreated','LastSeen'], axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             354369 non-null  int64 
 1   VehicleType       316879 non-null  object
 2   RegistrationYear  354369 non-null  int64 
 3   Gearbox           334536 non-null  object
 4   Power             354369 non-null  int64 
 5   Model             334664 non-null  object
 6   Kilometer         354369 non-null  int64 
 7   FuelType          321474 non-null  object
 8   Brand             354369 non-null  object
 9   NotRepaired       283215 non-null  object
dtypes: int64(4), object(6)
memory usage: 27.0+ MB


In [ ]:
# Мы заметили ,что в столбце RegistrationYear есть аномальные даты:
print(df[df['RegistrationYear']<1950]['RegistrationYear'].count())
print(df[df['RegistrationYear']>2016]['RegistrationYear'].count())

246
14530


In [ ]:
print(df[df['RegistrationYear']<1950]['RegistrationYear'].count()/len(df)*100)
print(df[df['RegistrationYear']>2016]['RegistrationYear'].count()/len(df)*100)

0.0694191647689273
4.100245788993958


In [ ]:
# Аномалььных дат примерно 4.17% от общего массива, но лучше удалить
df = df.drop(df[df['RegistrationYear']<1950].index).reset_index(drop = True)
df = df.drop(df[df['RegistrationYear']>2016].index).reset_index(drop = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339593 entries, 0 to 339592
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             339593 non-null  int64 
 1   VehicleType       316753 non-null  object
 2   RegistrationYear  339593 non-null  int64 
 3   Gearbox           321781 non-null  object
 4   Power             339593 non-null  int64 
 5   Model             322060 non-null  object
 6   Kilometer         339593 non-null  int64 
 7   FuelType          312434 non-null  object
 8   Brand             339593 non-null  object
 9   NotRepaired       274915 non-null  object
dtypes: int64(4), object(6)
memory usage: 25.9+ MB


In [ ]:
df['Car_age'] = 2016 - df['RegistrationYear']
df = df.drop('RegistrationYear', axis = 1)
df.describe()

,Price,Power,Kilometer,Car_age
count,339593.000000,339593.000000,339593.000000,339593.000000
mean,4471.566207,111.045696,128123.960152,13.474821
std,4545.245169,186.899948,37842.413937,6.835767
min,0.000000,0.000000,5000.000000,0.000000
25%,1100.000000,69.000000,125000.000000,9.000000
50%,2799.000000,105.000000,150000.000000,13.000000
75%,6500.000000,143.000000,150000.000000,17.000000
max,20000.000000,20000.000000,150000.000000,66.000000


In [ ]:
df.head()

,Price,VehicleType,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,Car_age
0,480,NaN,manual,0,golf,150000,petrol,volkswagen,NaN,23
1,18300,coupe,manual,190,NaN,125000,gasoline,audi,yes,5
2,9800,suv,auto,163,grand,125000,gasoline,jeep,NaN,12
3,1500,small,manual,75,golf,150000,petrol,volkswagen,no,15
4,3600,small,manual,69,fabia,90000,gasoline,skoda,no,8


In [ ]:
# Посмотрим на наименования брендов
df['Brand'].unique()

array(['volkswagen', 'audi', 'jeep', 'skoda', 'bmw', 'peugeot', 'ford',
       'mazda', 'nissan', 'renault', 'mercedes_benz', 'seat', 'honda',
       'fiat', 'opel', 'mini', 'smart', 'hyundai', 'sonstige_autos',
       'alfa_romeo', 'subaru', 'volvo', 'mitsubishi', 'kia', 'suzuki',
       'lancia', 'citroen', 'toyota', 'chevrolet', 'dacia', 'daihatsu',
       'trabant', 'saab', 'chrysler', 'jaguar', 'daewoo', 'porsche',
       'rover', 'land_rover', 'lada'], dtype=object)

In [ ]:
# Имеется странная марка Sonstige_autos, проверим, сколько по ней позиций в массиве
df[df['Brand']=='sonstige_autos'].isna().sum()

Price             0
VehicleType     834
Gearbox         963
Power             0
Model          3135
Kilometer         0
FuelType        946
Brand             0
NotRepaired    1176
Car_age           0
dtype: int64

In [ ]:
# Видим, что у данной марки нет ни одной модели, эти данные составляют менее 1%, поэтому можно удалить эти строки
df = df.drop(df[df['Brand']=='sonstige_autos'].index).reset_index(drop = True)
df.head(10)

,Price,VehicleType,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,Car_age
0,480,NaN,manual,0,golf,150000,petrol,volkswagen,NaN,23
1,18300,coupe,manual,190,NaN,125000,gasoline,audi,yes,5
2,9800,suv,auto,163,grand,125000,gasoline,jeep,NaN,12
3,1500,small,manual,75,golf,150000,petrol,volkswagen,no,15
4,3600,small,manual,69,fabia,90000,gasoline,skoda,no,8
5,650,sedan,manual,102,3er,150000,petrol,bmw,yes,21
6,2200,convertible,manual,109,2_reihe,150000,petrol,peugeot,no,12
7,0,sedan,manual,50,other,40000,petrol,volkswagen,no,36
8,14500,bus,manual,125,c_max,30000,petrol,ford,NaN,2
9,999,small,manual,101,golf,150000,NaN,volkswagen,NaN,18


In [ ]:
# Рассмотрим аномалии в столбце мощности мотора, максимальное значение значится 20тыс лс
df[df['Power']>500]['Power'].count()

410

In [ ]:
df[df['Power']==0]['Power'].count()

35101

Будем считать, что авто с нулевой мощностью это авто без двигателя - выставлены на продажу запчастей

In [ ]:
df[df['Power']>500]['Model'].unique()
# очень сомнительно, что Toyota Corolla под капотом имеет 500лс или Fabia

array(['corolla', 'vectra', 'colt', 'astra', 'matiz', 'punto', 'corsa',
       'lupo', nan, 'ypsilon', 'zafira', 'focus', 'other', 'ka', 's_type',
       'escort', 'm_reihe', 'ibiza', '5er', 'micra', 'e_klasse',
       'cayenne', 'c5', 'polo', '3er', 'fabia', 'caddy', 'golf', 'v40',
       'a8', 'aygo', '3_reihe', 'mondeo', 'arosa', 'c4', 'a6', 'twingo',
       'fortwo', 'a4', 'seicento', 'fiesta', 'leon', 'c_klasse',
       'berlingo', 'omega', 'sprinter', 'rio', 'scenic', 'a_klasse',
       'passat', 'ptcruiser', '6_reihe', 'touran', '1er', 'stilo',
       's_klasse', 'megane', 'transporter', 'galaxy', 'ceed', '80',
       'doblo', 'octavia', 'clk', 'cooper', 'cordoba', '147', 'agila',
       'outlander', 'mustang', 'sharan', 'm_klasse', 'kangoo', '500',
       'civic', 'a3', 'forfour', 'rav', 'laguna', 'bravo', 'spark',
       'nubira', 'kaefer', '2_reihe', 'espace', 'toledo', 'tt', 'up',
       'c2', 'touareg', 'clio', 'tiguan', 'c_max', 'yaris', 'cl',
       'x_trail', 'carisma'],

In [ ]:
# 410 строк, где мощность мотора свыше 500лс, можно смело удалять
df = df.drop(df[df['Power']>500].index).reset_index(drop = True)
df.describe()

,Price,Power,Kilometer,Car_age
count,336048.000000,336048.000000,336048.000000,336048.000000
mean,4464.490638,107.680251,128469.385326,13.399193
std,4533.204319,62.335755,37430.104124,6.666457
min,0.000000,0.000000,5000.000000,0.000000
25%,1100.000000,69.000000,125000.000000,9.000000
50%,2799.000000,105.000000,150000.000000,13.000000
75%,6500.000000,143.000000,150000.000000,17.000000
max,20000.000000,500.000000,150000.000000,66.000000


Выводы: Мы почистили таблицу от аномальных данных:
- строки, где мощность моторов выше 500лс;
- строки, где год регистрации указан больше 2016 года и меньше 1950 года;
- строки, где не было указано ни модели машины, ни бренда;
и удалили ненужные признаки в целом.

In [ ]:
df[df.duplicated()==True]

,Price,VehicleType,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,Car_age
1052,950,small,manual,60,polo,150000,petrol,volkswagen,no,17
1316,0,NaN,NaN,0,NaN,150000,NaN,volvo,NaN,16
2051,0,coupe,manual,129,c_klasse,150000,petrol,mercedes_benz,no,14
2676,1499,small,manual,58,corsa,150000,petrol,opel,no,15
2860,2800,wagon,NaN,0,touran,150000,gasoline,volkswagen,no,11
...,...,...,...,...,...,...,...,...,...,...
336032,6500,sedan,auto,145,e_klasse,150000,gasoline,mercedes_benz,no,13
336035,4400,sedan,manual,105,leon,150000,gasoline,seat,no,8
336038,1490,small,manual,50,lupo,150000,petrol,volkswagen,no,18
336039,7900,sedan,manual,140,golf,150000,gasoline,volkswagen,no,6


## Функция кодирование категориальных признаков

In [ ]:
def enum_column(data, column):
    '''
    Функция принимает на вход массив data и название колонки в массиве column, которую будем кодировать.
    Создаем словарь с нумерацией, испоьзуя функцию enumerate.
    Подставляем цифры вместо названий в ячейках через метод map.
    Возвращаем массив data и, если был пропуск, то число, заменяющее пропуск
    '''
    col_dct = dict(enumerate(data[column].unique()))
    col_dict = {v:k for k,v in col_dct.items()}
    if 'unknown' in col_dict:
        un_known = col_dict["unknown"]
        data[column] = data[column].map(col_dict)
        return data, un_known
    else:
        data[column] = data[column].map(col_dict)
        return data

## Заполнение пропусков модели

In [ ]:
# Функция модели предсказания пропуска

def knn_clf(X_train, y_train, X_test):
    knn_model = KNeighborsClassifier()
    knn_model.fit(X_train, y_train)
    return knn_model.predict(X_test)

In [ ]:
# С помощью модели Ближайших соседей заполним пропуски.

for brand in df['Brand'].unique():
    brand_df = df[df['Brand']==brand][['Price','Car_age','Power','Model','Kilometer']]
    brand_df_zeros = brand_df[brand_df['Model'].isna()==True]
    brand_df_train = brand_df[brand_df['Model'].isna()==False]
    
    
    model_dct = dict(enumerate(brand_df_train['Model'].unique()))
    model_dict = {v:k for k,v in model_dct.items()}
    brand_df_train['Model'] = brand_df_train['Model'].map(model_dict)
    
    try:
        X_train = brand_df_train.drop(['Model'], axis = 1)
        y_train = brand_df_train['Model']
        X_test = brand_df_zeros.drop(['Model'], axis = 1)
        brand_df_zeros['Model'] = knn_clf(X_train, y_train, X_test)
        brand_df_zeros['model'] = brand_df_zeros['Model'].map(model_dct)
        df['Model'].iloc[brand_df_zeros['model'].index] = np.array(brand_df_zeros['model'])
    except:
        pass    

In [ ]:
df['Model'].isna().sum()

0

In [ ]:
df = enum_column(df, 'Model')

In [ ]:
# Заполним пропуски словом unknown, чтобы удобно было применить 
df = df.fillna('unknown')

In [ ]:
df.head()

,Price,VehicleType,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,Car_age
0,480,unknown,manual,0,0,150000,petrol,volkswagen,unknown,23
1,18300,coupe,manual,190,1,125000,gasoline,audi,yes,5
2,9800,suv,auto,163,2,125000,gasoline,jeep,unknown,12
3,1500,small,manual,75,0,150000,petrol,volkswagen,no,15
4,3600,small,manual,69,3,90000,gasoline,skoda,no,8


## Кодирование столбцов

In [ ]:
df = enum_column(df, 'Brand')

In [ ]:
for column in ['VehicleType', 'FuelType', 'Gearbox']:
    df, un_known = enum_column(df, column)
    X_train = df.loc[df[column]!=un_known, ['Model', 'Car_age', 'Brand']]
    y_train = df.loc[df[column]!=un_known, column]
    X_test = df.loc[df[column]==un_known, ['Model', 'Car_age', 'Brand']]
    y_test = knn_clf(X_train, y_train, X_test)
    df.loc[df[column]==un_known, column] = y_test

In [ ]:
df = enum_column(df, 'NotRepaired')[0]

In [ ]:
df.head()

,Price,VehicleType,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,Car_age
0,480,3,0,0,0,150000,0,0,0,23
1,18300,1,0,190,1,125000,1,1,1,5
2,9800,2,1,163,2,125000,1,2,0,12
3,1500,3,0,75,0,150000,0,0,2,15
4,3600,3,0,69,3,90000,1,3,2,8


## Обучение моделей

In [ ]:
# Поделим наш массив на выборки для обучения моделей
X = df.drop('Price', axis = 1)
y = df['Price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
numeric_features = ['Power', 'Kilometer', 'Car_age']
categ_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

In [ ]:
# Создадим списки данных для наших моделей: Линейная регрессия, Лес и LGBM
names = [
    'LinearRegression',
    'RandomForestRegressor',
    'LGBMRegressor',
    'CatBoostRegressor'
]

models = [
    LinearRegression(),
    RandomForestRegressor(),
    LGBMRegressor(),
    CatBoostRegressor()
]

parameters = [
    {},
    {'model__n_estimators':[101],
     'model__max_depth':[19],
     'model__random_state':[0]
    },
    {'model__max_depth':[12],
     'model__learning_rate' : [0.4],
    'model__n_estimators':[111],
     'model__random_state':[0]
    },
    {'model__iterations':[200],
     'model__learning_rate':[0.4],
     'model__max_depth':[12],
     'model__random_seed':[0],
     'model__loss_function':['RMSE']
     }
]

In [ ]:
def model_train(model, param_grid):
    '''
    Функция принимает на вход модель и набор гиперпараметров,
    подбирает наилучшие гиперпараметры через GridSearchCV библиотеку, где предварительно
    колонки Power, Kilometer и car_age масштабируются
    '''
    preprocessor = ColumnTransformer(transformers = [('num', StandardScaler(), numeric_features)],
                                     remainder = 'passthrough')
    clf = Pipeline(steps = [('model', model),
                        ('model', model)])
    start_time = time.time()
    grid = GridSearchCV(clf, param_grid, cv= 5, n_jobs = -1, scoring = 'neg_mean_squared_error').fit(X_train, y_train)
    
    train_time = round(time.time()-start_time,2)
    #return grid.best_estimator_, grid.best_params_
    return grid.best_estimator_, train_time

In [ ]:
# Создадим таблицу, где будем сохранять результаты обучения и предсказания моделей
column = ['Модель', 'Гиперпараметры', 'Время обучения, сек', 'Время предсказания, сек', 'Метрика RMSE, евро']
results = pd.DataFrame(columns=column)

In [ ]:
# В цикле обучим наши модели из списка, посчитаем время предсказания и метрику RMSE, сохраним результаты в таблице
for name, model, param_grid in zip(names,models, parameters):
    model, train_time = model_train(model, param_grid)
    model.fit(X_train, y_train)
    start_time = time.time()
    y_pred = model.predict(X_test)
    predict_time = round(time.time()-start_time, 2)
    rmse = mean_squared_error(y_test, y_pred)**0.5
    results = pd.concat([results, pd.DataFrame([[name, model, train_time, predict_time, rmse]], columns = column)])

0:	learn: 3308.0335720	total: 233ms	remaining: 46.4s
1:	learn: 2666.2695613	total: 358ms	remaining: 35.4s
2:	learn: 2338.2945817	total: 483ms	remaining: 31.7s
3:	learn: 2130.9396499	total: 605ms	remaining: 29.6s
4:	learn: 2039.0243890	total: 730ms	remaining: 28.5s
5:	learn: 1961.7731182	total: 859ms	remaining: 27.8s
6:	learn: 1920.4471564	total: 987ms	remaining: 27.2s
7:	learn: 1893.4743153	total: 1.11s	remaining: 26.7s
8:	learn: 1862.7207885	total: 1.23s	remaining: 26.2s
9:	learn: 1844.8323838	total: 1.36s	remaining: 25.8s
10:	learn: 1828.0797352	total: 1.48s	remaining: 25.5s
11:	learn: 1815.1853175	total: 1.61s	remaining: 25.2s
12:	learn: 1799.6379193	total: 1.73s	remaining: 24.9s
13:	learn: 1790.4712868	total: 1.86s	remaining: 24.7s
14:	learn: 1783.2081584	total: 1.98s	remaining: 24.4s
15:	learn: 1769.1601767	total: 2.11s	remaining: 24.3s
16:	learn: 1758.3818161	total: 2.23s	remaining: 24s
17:	learn: 1753.0956709	total: 2.36s	remaining: 23.9s
18:	learn: 1747.7310348	total: 2.48s	rem

In [ ]:
results

,Модель,Гиперпараметры,"Время обучения, сек","Время предсказания, сек","Метрика RMSE, евро"
0,LinearRegression,"(ColumnTransformer(remainder='passthrough',\n ...",2.56,0.02,3144.118692
0,RandomForestRegressor,"(ColumnTransformer(remainder='passthrough',\n ...",309.46,2.36,1676.311629
0,LGBMRegressor,"(ColumnTransformer(remainder='passthrough',\n ...",10.01,0.23,1701.669542
0,CatBoostRegressor,"(ColumnTransformer(remainder='passthrough',\n ...",139.95,0.08,1658.281026


## Dummy Regression

In [ ]:
# Обучим Dummy модельб выбрав медиану как стратегию предсказания
dum = DummyRegressor(strategy = 'median').fit(X_train, y_train)
y_pred = dum.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred)**0.5

4802.361835538281

## Анализ моделей

Мы провели сравнительный анализ метрик моделей разных категорий: референсная Dummy модель, Линейная регрессия, Лес и модель градиентного спуска LGBM. По результатам исследований выявили, что погрешность предскзаания Леса ниже (1679 евро), чем у других моделей, но требуется заметно больше времени на обучение (352 секунда) и на предсказание (2,22 секунды),модель градиентного спуска LGBM обучается быстрей (86 секунды) и быстрей предсказывает (менее 1 секунды), но точность на 100 евро хуже (1780 евро), чем у Леса.

При условии, что Заказчику важны все 3 параметра (точность предсказания, время предсказания и время обучения) для определения стоимости, то считаю, что можно использовать модель LGBM.